In [ ]:
import os
import json
import requests
from dotenv import load_dotenv

In [ ]:
# === Azure Foundry Setup ===
API_URL = os.getenv("AZEV_API_URL")
API_KEY = os.getenv("AZEV_API_KEY")
MODEL_ID = os.getenv("AZEV_MODEL_ID")

In [ ]:
HEADERS = {
    "Authorization": f"Bearer {API_KEY}",
    "Content-Type": "application/json"
}

In [ ]:
debug = False
epochs = 10

In [ ]:

# === Process Each File ===
def process_files(folder_path, file_prefix):

    for filename in os.listdir(folder_path):
        if filename.startswith(file_prefix):
            file_path = os.path.join(folder_path, filename)

            if os.path.isfile(file_path):
                with open(file_path, "r", encoding="utf-8") as f:
                    completion_old = json.load(f)
                    generation_old = completion_old.get("generation", "No prompt provided.")

                payload = {
                    "model": MODEL_ID,
                    "messages": [
                        {"role": "system", "content": "You are an intelligent assistant."},
                        {"role": "user", "content":
                            f"Evaluate the following generated text by checking the Coherence, Consistency, Grammar, Syntax, Spelling" +
                            f"Try to be as concise as possible, so everytime the score will be the same." +
                            f"Always provide the total-score at the top and sub-score inline with explanation." +
                            f"The score should be an integer number between 1 and 100, where 1 is the worst and 100 is the best." +
                            f"Output only the integer score number" +
                            f"\n\n"
                            f"\"{generation_old}\""
                         }
                    ]
                }

                for i in range(epochs):
                    score_new = 0
                    score_cnt = 0
                    generation_new = ""
                    response = requests.post(url=API_URL, headers=HEADERS, json=payload)
                    response_json = response.json()
                    try:
                        generation_new = response_json["choices"][0]["message"]["content"]
                        score_cnt += 1
                        score_new += int(generation_new.split()[0])
                        if debug:
                            print(f"\n📄 {filename}\nModel Output:\n{generation_new}")
                    except (KeyError, IndexError, ValueError):
                        if debug:
                            print(f"\n📄 {filename}\nModel Output:\n{response_json}")

                score_avg = score_new // score_cnt if score_cnt > 0 else 0
                with open(file_path, "w", encoding="utf-8") as f:
                    
                    print(f"\n📄 {filename}\nEvaluation:\n{score_avg}")
                    completion_old["evaluation"] = score_avg
                    completion_old = json.dump(completion_old, f, indent=4)

In [ ]:
# === Run the script ===
if __name__ == "__main__":
    folder_path = "../outputs"
    file_prefix = "completion_"
    process_files(folder_path, file_prefix)